In [15]:
import pandas as pd

In [189]:
import os
import re
import collections
from collections import Counter, defaultdict
from itertools import chain
import time
import json

In [17]:
ls

WOS Cleaning and Cofunder Data Processing.ipynb
plotting.R
top_cofunder_frequencies.csv
top_cofunder_frequencies_validated_06182021.xlsx
wos_funders_2010to2021.xlsx
~$top_cofunder_frequencies_validated_06182021.xlsx


In [18]:
df = pd.read_excel("wos_funders_2010to2021.xlsx")

In [19]:
df.head()

,Unnamed: 0,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,...,IDS Number,UT (Unique WOS ID),Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,Unnamed: 67,funder_dataset,oa_summary
0,0,J,"He, QW; Borgonovi, F; Paccagnella, M",NaN,NaN,NaN,"He, Qiwei; Borgonovi, Francesca; Paccagnella, ...",NaN,NaN,Leveraging process data to assess adults' prob...,...,QT0TV,WOS:000626306200014,NaN,NaN,NaN,NaN,2021-06-14,NaN,osf,NO_OA
1,1,J,"Misoska, AT; Loader, R",NaN,NaN,NaN,"Misoska, Ana Tomovska; Loader, Rebecca",NaN,NaN,The role of school-based contact in reducing s...,...,SG2DR,WOS:000653253800001,NaN,NaN,NaN,NaN,2021-06-14,NaN,osf,PUBLISHER_HOSTED
2,2,J,"Chin, ET; Leidner, D; Ryckman, T; Liu, YRE; Pr...",NaN,NaN,NaN,"Chin, Elizabeth T.; Leidner, David; Ryckman, T...",NaN,NaN,Covid-19 Vaccine Acceptance in California Stat...,...,SB2FY,WOS:000649816900001,33979505.0,"Bronze, Green Published",NaN,NaN,2021-06-14,NaN,osf,AUTHOR_HOSTED
3,3,J,"Swencionis, JK; Pouget, ER; Goff, PA",NaN,NaN,NaN,"Swencionis, Jillian K.; Pouget, Enrique R.; Go...",NaN,NaN,Supporting social hierarchy is associated with...,...,RX4ME,WOS:000647198300019,NaN,NaN,NaN,NaN,2021-06-14,NaN,osf,NO_OA
4,4,J,"Greene, SK; Peterson, ER; Balan, D; Jones, L; ...",NaN,NaN,NaN,"Greene, Sharon K.; Peterson, Eric R.; Balan, D...",NaN,NaN,Detecting COVID-19 Clusters at High Spatiotemp...,...,SB6SQ,WOS:000650121700033,33900181.0,"DOAJ Gold, Green Published",NaN,NaN,2021-06-14,NaN,osf,NO_OA


In [96]:
funders = list(set(df.funder_dataset))
funders

['mellon', 'packard', 'gates', 'osf', 'rwj', 'hewlett', 'ford']

In [21]:
# remove rows that got thrown into the mellon query based on the Robert King Mellon Foundation
# remove rows that include carnegie or paul mellon without andrew or a.w.
df["remove_rows"] = [1 if (dataset=="mellon" and any(ele in funderString.lower() for ele in ["r. k.","r.k.","king","richard k"])) else 0 for (dataset,funderString) in zip(df['funder_dataset'],df['Funding Orgs']) ]
df["remove_rows"] = [1 if (dataset=="mellon" and any(ele in funderString.lower() for ele in ["carnegie","paul"]) and not(any(ele in funderString.lower() for ele in ["a.w.","andrew"]))) else cur_val for (dataset,funderString,cur_val) in zip(df['funder_dataset'],df['Funding Orgs'],df['remove_rows'])]

In [22]:
df['Funding Orgs'] = [x.replace("AMP;","&") for x in df['Funding Orgs']]

In [23]:
df.columns

Index(['Unnamed: 0', 'Publication Type', 'Authors', 'Book Authors',
       'Book Editors', 'Book Group Authors', 'Author Full Names',
       'Book Author Full Names', 'Group Authors', 'Article Title',
       'Source Title', 'Book Series Title', 'Book Series Subtitle', 'Language',
       'Document Type', 'Conference Title', 'Conference Date',
       'Conference Location', 'Conference Sponsor', 'Conference Host',
       'Author Keywords', 'Keywords Plus', 'Abstract', 'Addresses',
       'Reprint Addresses', 'Email Addresses', 'Researcher Ids', 'ORCIDs',
       'Funding Orgs', 'Funding Text', 'Cited References',
       'Cited Reference Count', 'Times Cited, WoS Core',
       'Times Cited, All Databases', '180 Day Usage Count',
       'Since 2013 Usage Count', 'Publisher', 'Publisher City',
       'Publisher Address', 'ISSN', 'eISSN', 'ISBN', 'Journal Abbreviation',
       'Journal ISO Abbreviation', 'Publication Date', 'Publication Year',
       'Volume', 'Issue', 'Part Number', 'Suppleme

In [24]:
sum(df.remove_rows)

266

In [25]:
df = df[df.remove_rows != 1]

In [26]:
df['grant_nums'] = [re.findall(r'\[[a-zA-Z0-9\-\.\s\:\,\/\\]*\]',str(x)) for x in df['Funding Orgs']]
df['cofunder_clean'] = df['Funding Orgs']
for index,row in df.iterrows():
    if len(row['grant_nums']) > 0:
        for gn in row['grant_nums']:
            df.at[index,'cofunder_clean'] = df.at[index,'cofunder_clean'].replace(gn,"")

In [27]:
df["cofunders_list"] = [x.split(";") for x in df['cofunder_clean']]

In [86]:
def make_cofunder_dict(df,funder_list_col):
    cofunderDict = {}
    for f in funders:
        d = df[df.funder_dataset == f]
        cof_list = list(chain.from_iterable(d[funder_list_col]))
        cof_list = [x.strip() for x in cof_list]
        cofunderDict[f] = collections.Counter(cof_list)
    return(cofunderDict)

In [88]:
cofunderDict = make_cofunder_dict(df,'cofunders_list')

In [89]:
cofunderDict['osf']

Counter({'Open Society Foundations through the OECD Thomas J. Alexander Fellowship Programme': 1,
         'National Science FoundationNational Science Foundation (NSF)': 69,
         'British Academy Global Professorship scheme': 1,
         'Atlantic Philanthropies': 7,
         'Open Society Foundations': 237,
         'Covid-19 Emergency Response Fund at Stanford': 1,
         'National Institute on Drug AbuseUnited States Department of Health & Human ServicesNational Institutes of Health (NIH) - USANIH National Institute on Drug Abuse (NIDA)European Commission': 35,
         'Centers for Disease Control and Prevention though the Council of State and Territorial EpidemiologistsUnited States Department of Health & Human ServicesCenters for Disease Control & Prevention - USA': 1,
         'National Science Foundation Graduate Research Fellowship ProgramNational Science Foundation (NSF)': 2,
         'Ford Foundation': 24,
         'Google.org via the Tides Foundation': 1,
         'K

In [55]:
## number of unique keys
for k,v in cofunderDict.items():
    print(k,len(list(set(v))))

mellon 5211
packard 13728
gates 31527
osf 2466
rwj 12227
hewlett 2005
ford 4574


## Text distance trials...

In [32]:
import textdistance

In [33]:
textdistance.ratcliff_obershelp("merry christmas mer","mer")

0.2727272727272727

In [34]:
len("mer")/len("merry christmas mer")

0.15789473684210525

In [98]:
## get the X most frequent cofunders, and let's try measuring substring similarity with other entries
## 

def get_freq_cofunders_similarities(df,cofunderDict,funder_list_col,funder_n_most,get_sims):
    cofSim ={}
    topCof = {}

    start = time.time()
    for f in funders:
        cofunders = [(x,y) for (x,y) in cofunderDict[f].most_common(n_most)]
        topCof[f] = cofunders
        if get_sims is True:
            d = df[df.funder_dataset == f]
            cof_list = list(chain.from_iterable(d[funder_list_col]))
            cof_list = [x.strip() for x in cof_list]
            cofSim[f] = {}
            for (x,y) in cofunders:
                similarities = []
                t1 = x.replace("Foundation","").replace("National","").strip()
                for cof in cof_list:
                    t2 = cof.replace("Foundation","").replace("National","").strip()
                    ## if there is at least some common element
                    if bool(set(t1.split(" ")) & set(t2.split(" "))):
                        similarities.append(tuple((tuple((x,cof)),textdistance.ratcliff_obershelp(t1,t2))))
                similarities = [(strings,measure) for (strings,measure) in similarities if measure > 0.32]   
                #similarities = [tuple((tuple((x,y)),textdistance.ratcliff_obershelp(x.replace("Foundation",""),y.replace("Foundation","")))) for y in cof_list  ]
                #similarities = [(strings,measure) for (strings,measure) in similarities if measure > 0.32]
                cofSim[f][x] = collections.Counter(similarities)
        print(time.time() - start)
    return(cofSim,topCof)
        
        
        

In [99]:
cofSim,topCof = get_freq_cofunders_similarities(df,cofunderDict,'cofunders_list',250,False)

0.0017240047454833984
0.005617856979370117
0.013762950897216797
0.014818191528320312
0.018159866333007812
0.019047021865844727
0.02035689353942871


In [94]:
cofSim

{}

In [100]:
topCof['osf']

[('Open Society Foundations', 237),
 ('Open Society Foundation', 112),
 ('Open Society Institute', 74),
 ('NIHUnited States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA',
  70),
 ('National Science FoundationNational Science Foundation (NSF)', 69),
 ('Paul and Daisy Soros Fellowship', 68),
 ('Paul and Daisy Soros Fellowship for New Americans', 65),
 ('NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCESUnited States Department of Health & Human ServicesNational Institutes of Health (NIH) - USANIH National Institute of General Medical Sciences (NIGMS)  Funding Source: NIH RePORTER',
  65),
 ('NATIONAL INSTITUTE ON DRUG ABUSEUnited States Department of Health & Human ServicesNational Institutes of Health (NIH) - USANIH National Institute on Drug Abuse (NIDA)European Commission  Funding Source: NIH RePORTER',
  53),
 ('NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS DISEASESUnited States Department of Health & Human ServicesNational Institutes of Health (NIH) - US

In [108]:
## output a dataframe of the top cofunders,
## cleanup in excel, for the top 25-50 for each org, use similarity to get cleaner counts
def get_cof_freq_df(topCofDict):
    cofDf = pd.DataFrame()
    funder_dataset = []
    entry = []
    val = []
    for k,v in topCofDict.items():
        for funder,count in v:
            funder_dataset.append(k)
            entry.append(funder)
            val.append(count)
            
    cofDf["dataset"] = funder_dataset
    cofDf["funder_value"] = entry
    cofDf["frequency"] = val
    return(cofDf)

In [109]:
cofDf = get_cof_freq_df(topCof)

In [110]:
cofDf

,dataset,funder_value,frequency
0,mellon,Andrew W. Mellon Foundation,1125
1,mellon,Mellon Foundation,536
2,mellon,National Science FoundationNational Science Fo...,485
3,mellon,Andrew Mellon Foundation,366
4,mellon,NSFNational Science Foundation (NSF),198
...,...,...,...
1745,ford,Paul B. Beeson Award in Aging,4
1746,ford,Starr Foundation,4
1747,ford,American Federation for Aging Research,4
1748,ford,AmgenAmgen,4


In [27]:
cofDf.to_csv("top_cofunder_frequencies.csv")

In [28]:
cofDf.head(15)

,dataset,funder_value,frequency
0,gates,Bill and Melinda Gates FoundationBill & Melind...,9873
1,gates,Bill & Melinda Gates FoundationBill & Melinda ...,5548
2,gates,NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS D...,3625
3,gates,Bill & Melinda Gates FoundationBill & Melinda ...,2579
4,gates,NIHUnited States Department of Health & Human ...,1538
5,gates,Wellcome TrustWellcome TrustEuropean Commission,1473
6,gates,National Institutes of HealthUnited States Dep...,1402
7,gates,Medical Research CouncilUK Research & Innovati...,899
8,gates,NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCES...,815
9,gates,EUNICE KENNEDY SHRIVER NATIONAL INSTITUTE OF C...,809


In [31]:
df[df['Funding Orgs'].str.contains("AMP")]['Funding Orgs']

378      HAMP;H Evergreen Foundation; NIBIBUnited State...
439      NIH from the National Cancer Institute (NCI) [...
677      National Institute on Drug AbuseUnited States ...
1662     National Institutes of Health, United StatesUn...
1857     Leona M. and Harry B. Helmsley Charitable Trus...
                               ...                        
47361    NCEAS; NSFNational Science Foundation (NSF) [E...
48231    National Institutes of Health (NIH)United Stat...
49054    CONACYTConsejo Nacional de Ciencia y Tecnologi...
49971    National Science FoundationNational Science Fo...
50003    National Institutes of Health, United StatesUn...
Name: Funding Orgs, Length: 495, dtype: object

## Test for Repeated Funder Strings and clean them

In [1]:
def principal_period(s):
    i = (s+s).find(s, 1, -1)
    return None if i == -1 else s[:i]


In [14]:
df = pd.read_clipboard()

EmptyDataError: No columns to parse from file

In [9]:
df

,dataset,funder_value,frequency,validated_funder
0,gates,Bill and Melinda Gates FoundationBill & Melind...,9873,Bill and Melinda Gates Foundation
1,gates,Bill & Melinda Gates FoundationBill & Melinda ...,5548,Bill and Melinda Gates Foundation
2,gates,NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS D...,3625,United States National Institutes of Health (NIH)
3,gates,Bill & Melinda Gates FoundationBill & Melinda ...,2579,Bill and Melinda Gates Foundation; CGIAR
4,gates,NIHUnited States Department of Health & Human ...,1538,United States National Institutes of Health (NIH)
...,...,...,...,...
1745,osf,Michael Smith Foundation for Health ResearchMi...,3,Michael Smith Foundation for Health Research
1746,osf,Livestrong Foundation,3,NaN
1747,osf,LIVESTRONG Foundation,3,NaN
1748,osf,Boehringer IngelheimBoehringer Ingelheim,3,NaN


In [11]:
df['dup_rep'] = [principal_period(str(x)) for x in df['funder_value']]

In [13]:
df.to_clipboard()

## Merge validated entries back in to a validated cofunder column

In [132]:
valDf = pd.read_excel("top_cofunder_frequencies_validated_06182021.xlsx",sheet_name="top_cofunder_frequencies")

In [133]:
valDf

,Unnamed: 0,dataset,funder_value,frequency,validated_funder,round
0,0,gates,Bill and Melinda Gates FoundationBill & Melind...,9873,Bill and Melinda Gates Foundation,NaN
1,1,gates,Bill & Melinda Gates FoundationBill & Melinda ...,5548,Bill and Melinda Gates Foundation,NaN
2,2,gates,NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS D...,3625,United States National Institutes of Health (NIH),NaN
3,3,gates,Bill & Melinda Gates FoundationBill & Melinda ...,2579,Bill and Melinda Gates Foundation; CGIAR,NaN
4,4,gates,NIHUnited States Department of Health & Human ...,1538,United States National Institutes of Health (NIH),NaN
...,...,...,...,...,...,...
1745,1494,osf,Michael Smith Foundation for Health ResearchMi...,3,Michael Smith Foundation for Health Research,NaN
1746,1495,packard,Wisconsin Alumni Research Foundation,25,Wisconsin Alumni Research Foundation,NaN
1747,1498,hewlett,Wolf Creek Charitable Foundation,4,Wolf Creek Charitable Foundation,NaN
1748,1499,osf,Boehringer IngelheimBoehringer Ingelheim,3,Boehringer Ingelheim,NaN


In [148]:
def merge_validated_entries(valDf,df,cofunders_list,validated_cof_list,full_merge=True):
    
    if full_merge is False:
        valDf = valDf[~valDf.validated_funder.isna() ]
    valDict = dict()
    for index,row in valDf.iterrows():
        valDict[str(row['funder_value']).strip()] = str(row['validated_funder']).strip()
        
    df[validated_cof_list] = ""
    for index,row in df.iterrows():
        validated_list = []
        for funder in row[cofunders_list]:
            validated_list.append(valDict.get(funder.strip(),funder).strip())
        df.at[index,validated_cof_list] = list(set(validated_list))
    return(valDict,df)

In [150]:
valDict,df = merge_validated_entries(valDf,df,'cofunders_list','validated_cof_list',full_merge=True)

In [136]:
df['cofunders_list'][4]

['Public Health Emergency Preparedness Cooperative Agreement - Centers for Disease Control and Prevention (CDC)United States Department of Health & Human ServicesCenters for Disease Control & Prevention - USA ',
 ' SaTScan Enhancements Project',
 ' CDC FoundationUnited States Department of Health & Human ServicesCenters for Disease Control & Prevention - USA',
 ' CDC ELC CARES ',
 ' Alfred P. Sloan FoundationAlfred P. Sloan Foundation',
 ' Open Society Foundations']

In [137]:
df['validated_cof_list'][4]

['Public Health Emergency Preparedness Cooperative Agreement - Centers for Disease Control and Prevention (CDC)United States Department of Health & Human ServicesCenters for Disease Control & Prevention - USA',
 'SaTScan Enhancements Project',
 'CDC ELC CARES',
 'Alfred P. Sloan Foundation',
 'CDC FoundationUnited States Department of Health & Human ServicesCenters for Disease Control & Prevention - USA',
 'Open Society Foundations']

In [60]:
valDict['Alfred P. Sloan FoundationAlfred P. Sloan Foundation']

'Alfred P. Sloan Foundation'

In [73]:
for f in df['cofunders_list'][4]:
    print("\n",f,"\n\n")
    print(valDict.get(f.strip(),None))


 Public Health Emergency Preparedness Cooperative Agreement - Centers for Disease Control and Prevention (CDC)United States Department of Health & Human ServicesCenters for Disease Control & Prevention - USA  


None

  SaTScan Enhancements Project 


None

  CDC FoundationUnited States Department of Health & Human ServicesCenters for Disease Control & Prevention - USA 


None

  CDC ELC CARES  


None

  Alfred P. Sloan FoundationAlfred P. Sloan Foundation 


Alfred P. Sloan Foundation

  Open Society Foundations 


Open Society Foundations


In [70]:
valDict.keys()

dict_keys(['Bill and Melinda Gates FoundationBill & Melinda Gates Foundation', 'Bill & Melinda Gates FoundationBill & Melinda Gates Foundation', 'NATIONAL INSTITUTE OF ALLERGY AND INFECTIOUS DISEASESUnited States Department of Health & Human ServicesNational Institutes of Health (NIH) - USANIH National Institute of Allergy & Infectious Diseases (NIAID)  Funding Source: NIH RePORTER', 'Bill & Melinda Gates FoundationBill & Melinda Gates FoundationCGIAR', 'NIHUnited States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA', 'Wellcome TrustWellcome TrustEuropean Commission', 'National Institutes of HealthUnited States Department of Health & Human ServicesNational Institutes of Health (NIH) - USA', 'Medical Research CouncilUK Research & Innovation (UKRI)Medical Research Council UK (MRC)European Commission  Funding Source: researchfish', 'NATIONAL INSTITUTE OF GENERAL MEDICAL SCIENCESUnited States Department of Health & Human ServicesNational Institutes of Healt

In [138]:
validatedCofDict = make_cofunder_dict(df,'validated_cof_list')

In [139]:
validatedCofDict

{'mellon': Counter({'Government of South Africa': 42,
          'Andrew W. Mellon Foundation': 2278,
          'South African Water Research Commission': 1,
          'Office of the Vice President for Research at the University of South Carolina': 1,
          'National Endowment for the Humanities': 19,
          'Digital Bridges for Humanistic Inquiry: A Grinnell College/University of Iowa Partnership - Andrew W. Mellon Foundation': 1,
          'Obermann Center for Advanced Studies (University of Iowa)': 1,
          'UI Digital Scholarship and Publishing Studio': 1,
          'Portland State University': 1,
          'United States National Science Foundation (NSF)': 837,
          'All Souls College Oxford': 1,
          'Center for Advanced Study in the Behavioral Sciences at Stanford': 1,
          'Mellon Foundation through SANBI (South African National Biodiversity Institute)': 1,
          'Akademie der Wissenschaften und der Literatur Mainz, Germany': 1,
          'Mellon Fo

In [104]:
cofSim,topCof_validated = get_freq_cofunders_similarities(df,validatedCofDict,'validated_cof_list',250,False)

0.0015611648559570312
0.005530118942260742
0.012340068817138672
0.013749122619628906
0.01681995391845703
0.017599105834960938
0.01884317398071289


In [140]:
topCof_validated['mellon']

[('Andrew W. Mellon Foundation', 2278),
 ('United States National Science Foundation (NSF)', 837),
 ('South African National Research Foundation', 250),
 ('MacArthur Foundation', 178),
 ('United States National Institutes of Health (NIH)', 162),
 ('Smithsonian Institution', 154),
 ('Gordon and Betty Moore Foundation', 95),
 ('David and Lucile Packard Foundation', 88),
 ('W.M. Keck Foundation', 73),
 ('Avatar Alliance Foundation', 60),
 ('University of the Witwatersrand', 57),
 ('Grantham Foundation for the Protection of the Environment', 54),
 ('Margaret A. Cargill Foundation', 52),
 ('Small World Institute Fund', 51),
 ('National Geographic Society', 51),
 ('Celera Foundation', 50),
 ('UK Research and Innovation (UKRI)', 47),
 ('Wenner-Gren Foundation', 44),
 ('Social Sciences Research Council (SSRC)', 44),
 ('Government of South Africa', 42),
 ('United States National Aeronautics & Space Administration (NASA)', 36),
 ('American Council of Learned Societies', 36),
 ('Wellcome Trust', 

In [141]:
### to do: parse the strings that still have a semicolon in them after validation
## so that these strings are counted towards the accurate funders

onceValidated_cofDf = get_cof_freq_df(topCof_validated)

In [142]:
onceValidated_cofDf['dup_rep'] = [principal_period(str(x)) for x in onceValidated_cofDf['funder_value']]


In [143]:
onceValidated_cofDf

,dataset,funder_value,frequency,dup_rep
0,mellon,Andrew W. Mellon Foundation,2278,None
1,mellon,United States National Science Foundation (NSF),837,None
2,mellon,South African National Research Foundation,250,None
3,mellon,MacArthur Foundation,178,None
4,mellon,United States National Institutes of Health (NIH),162,None
...,...,...,...,...
1745,ford,MacArthur Foundation Grant,3,None
1746,ford,Ford Foundation's Predoctoral Fellowship,3,None
1747,ford,"University of Alabama at Birmingham, Departmen...",3,None
1748,ford,NIDA/NIHUnited States Department of Health & H...,3,None


In [114]:
onceValidated_cofDf.to_csv("validation_round_two.csv")

In [124]:
twiceValidatedDf = pd.read_csv("validation_round_two.csv")
twiceValidatedDf

,Unnamed: 0,dataset,funder_value,frequency,dup_rep,validated_funder
0,0,mellon,Andrew W. Mellon Foundation,2278,NaN,NaN
1,1,mellon,United States National Science Foundation (NSF),837,NaN,United States National Science Foundation (NSF)
2,2,mellon,South African National Research Foundation,250,NaN,NaN
3,3,mellon,MacArthur Foundation,178,NaN,NaN
4,4,mellon,United States National Institutes of Health (NIH),162,NaN,United States National Institutes of Health (NIH)
...,...,...,...,...,...,...
1745,1745,ford,MacArthur Foundation Grant,3,NaN,NaN
1746,1746,ford,Ford Foundation's Predoctoral Fellowship,3,NaN,NaN
1747,1747,ford,"University of Alabama at Birmingham, Departmen...",3,NaN,NaN
1748,1748,ford,NIDA/NIHUnited States Department of Health & H...,3,NaN,United States National Institutes of Health (NIH)


In [147]:
twiceValidatedDf[~twiceValidatedDf.validated_funder.isna()]

,Unnamed: 0,dataset,funder_value,frequency,dup_rep,validated_funder
1,1,mellon,United States National Science Foundation (NSF),837,NaN,United States National Science Foundation (NSF)
4,4,mellon,United States National Institutes of Health (NIH),162,NaN,United States National Institutes of Health (NIH)
56,500,gates,Bill and Melinda Gates Foundation,17030,NaN,Bill and Melinda Gates Foundation
93,93,mellon,Bill and Melinda Gates Foundation,7,NaN,Bill and Melinda Gates Foundation
171,171,mellon,British Heart FoundationBritish Heart Foundation,5,British Heart Foundation,British Heart Foundation
...,...,...,...,...,...,...
1697,1697,ford,US National Academy of Science,4,NaN,United States National Academy of Sciences
1704,1704,ford,National Institute of Biomedical Imaging and B...,4,NaN,United States National Institutes of Health (NIH)
1716,1716,ford,National Institute of General Medical Sciences...,3,NaN,United States National Institutes of Health (NIH)
1738,1738,ford,NIH-NCIUnited States Department of Health & Hu...,3,NaN,United States National Institutes of Health (NIH)


In [125]:
df

,Unnamed: 0,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,...,Hot Paper Status,Date of Export,Unnamed: 67,funder_dataset,oa_summary,remove_rows,grant_nums,cofunder_clean,cofunders_list,validated_cof_list
0,0,J,"He, QW; Borgonovi, F; Paccagnella, M",NaN,NaN,NaN,"He, Qiwei; Borgonovi, Francesca; Paccagnella, ...",NaN,NaN,Leveraging process data to assess adults' prob...,...,NaN,2021-06-14,NaN,osf,NO_OA,0,[[IIS-1633353]],Open Society Foundations through the OECD Thom...,[Open Society Foundations through the OECD Tho...,[Open Society Foundations through the OECD Tho...
1,1,J,"Misoska, AT; Loader, R",NaN,NaN,NaN,"Misoska, Ana Tomovska; Loader, Rebecca",NaN,NaN,The role of school-based contact in reducing s...,...,NaN,2021-06-14,NaN,osf,PUBLISHER_HOSTED,0,[[IN2015-23584]],Atlantic Philanthropies; Open Society Foundati...,"[Atlantic Philanthropies, Open Society Founda...","[Atlantic Philanthropies, Open Society Foundat..."
2,2,J,"Chin, ET; Leidner, D; Ryckman, T; Liu, YRE; Pr...",NaN,NaN,NaN,"Chin, Elizabeth T.; Leidner, David; Ryckman, T...",NaN,NaN,Covid-19 Vaccine Acceptance in California Stat...,...,NaN,2021-06-14,NaN,osf,AUTHOR_HOSTED,0,"[[R37-DA15612], [NU38OT000297-02], [DGE-165651...",Covid-19 Emergency Response Fund at Stanford; ...,"[Covid-19 Emergency Response Fund at Stanford,...",[United States National Institutes of Health (...
3,3,J,"Swencionis, JK; Pouget, ER; Goff, PA",NaN,NaN,NaN,"Swencionis, Jillian K.; Pouget, Enrique R.; Go...",NaN,NaN,Supporting social hierarchy is associated with...,...,NaN,2021-06-14,NaN,osf,NO_OA,0,"[[TFR1701409], [P3025019], [1620493, 1329162, ...",Atlantic Philanthropies; Ford Foundation; Goog...,"[Atlantic Philanthropies, Ford Foundation, G...","[Ford Foundation, Google.org via the Tides Fou..."
4,4,J,"Greene, SK; Peterson, ER; Balan, D; Jones, L; ...",NaN,NaN,NaN,"Greene, Sharon K.; Peterson, Eric R.; Balan, D...",NaN,NaN,Detecting COVID-19 Clusters at High Spatiotemp...,...,NaN,2021-06-14,NaN,osf,NO_OA,0,"[[NU90TP922035-01], [NU50CK000517-01-09]]",Public Health Emergency Preparedness Cooperati...,[Public Health Emergency Preparedness Cooperat...,[Public Health Emergency Preparedness Cooperat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50304,495,J,"Grine, FE",NaN,NaN,NaN,"Grine, Frederick E.",NaN,NaN,The alpha taxonomy of Australopithecus at Ster...,...,NaN,2021-06-14,NaN,ford,PUBLISHER_HOSTED,0,[],"College of Arts and Sciences, Stony Brook Univ...","[College of Arts and Sciences, Stony Brook Uni...","[Ford Foundation, University of the Witwatersr..."
50305,496,J,"Colozza, D; Avendano, M",NaN,NaN,NaN,"Colozza, David; Avendano, Mauricio",NaN,NaN,"Urbanisation, dietary change and traditional f...",...,NaN,2021-06-14,NaN,ford,NO_OA,0,"[[2R01 HD050764-05A1], [2R01 AG026676-05]]",IFLS 1: the National Institute for Child Healt...,[IFLS 1: the National Institute for Child Heal...,"[Ford Foundation, United States Agency for Int..."
50306,497,J,"Thomas, MD; Michaels, EK; Reeves, AN; Okoye, U...",NaN,NaN,NaN,"Thomas, Marilyn D.; Michaels, Elizabeth K.; Re...",NaN,NaN,Differential associations between everyday ver...,...,NaN,2021-06-14,NaN,ford,AUTHOR_HOSTED,0,"[[UL1GM118985], [GTDR14301469], [P60MD006902],...","University of California, Berkeley Hellman Fun...","[University of California, Berkeley Hellman Fu...","[Ford Foundation, NIMHD, USA, UCB Research Bri..."
50307,498,J,"Tanner, JC; Swanger, E; Zuk, M",NaN,NaN,NaN,"Tanner, Jessie C.; Swanger, Elizabeth; Zuk, Ma...",NaN,NaN,Sexual signal loss in field crickets maintaine...,...,NaN,2021-06-14,NaN,ford,PUBLISHER_HOSTED,0,[[00039202]],Theodore J. Cohn Research Grant from the Ortho...,[Theodore J. Cohn Research Grant from the Orth...,"[Ford Foundation, National Science Foundation ..."


In [151]:
twiceValDict, df = merge_validated_entries(twiceValidatedDf,df,'validated_cof_list','twice_validated_cof_list',full_merge=False)


In [152]:
twiceValDict

{'United States National Science Foundation (NSF)': 'United States National Science Foundation (NSF)',
 'United States National Institutes of Health (NIH)': 'United States National Institutes of Health (NIH)',
 'Bill and Melinda Gates Foundation': 'Bill and Melinda Gates Foundation',
 'British Heart FoundationBritish Heart Foundation': 'British Heart Foundation',
 'International Dissertation Research Fellowship Program of the Social Science Research Council': 'Danish Natural Science Research Council',
 'Table Mountain Fund': 'Research Corporation for Science Advancement',
 'US National Park Service': 'Institute for Advanced Study',
 'Historical Society of Pennsylvania': 'March of Dimes',
 'Chiang Ching-Kuo Foundation': 'University of Florida',
 'Rutgers University': 'Natural Sciences and Engineering Research Council of Canada (NSERC)',
 'NICHD through the University of Michigan': 'Novo Nordisk Foundation',
 'American Chemical Society Petroleum Research FundAmerican Chemical Society': '

In [155]:
twiceValidatedCofDict = make_cofunder_dict(df,'twice_validated_cof_list')
cofSim,topCof_twiceValidated = get_freq_cofunders_similarities(df,twiceValidatedCofDict,'twice_validated_cof_list',100,False)
twiceValidated_cofDf = get_cof_freq_df(topCof_twiceValidated)

0.0033631324768066406
0.006299257278442383
0.012648344039916992
0.013658285140991211
0.01631307601928711
0.017115116119384766
0.018168210983276367


In [156]:
topCof_twiceValidated

{'mellon': [('Andrew W. Mellon Foundation', 2290),
  ('United States National Science Foundation (NSF)', 845),
  ('South African National Research Foundation', 250),
  ('MacArthur Foundation', 178),
  ('United States National Institutes of Health (NIH)', 164),
  ('Smithsonian Institution', 154),
  ('Gordon and Betty Moore Foundation', 95),
  ('David and Lucile Packard Foundation', 88),
  ('W.M. Keck Foundation', 73),
  ('Avatar Alliance Foundation', 60),
  ('University of the Witwatersrand', 57),
  ('Grantham Foundation for the Protection of the Environment', 54),
  ('Margaret A. Cargill Foundation', 52),
  ('Small World Institute Fund', 51),
  ('National Geographic Society', 51),
  ('Celera Foundation', 50),
  ('UK Research and Innovation (UKRI)', 47),
  ('Wenner-Gren Foundation', 44),
  ('Social Sciences Research Council (SSRC)', 44),
  ('Government of South Africa', 42),
  ('United States National Aeronautics & Space Administration (NASA)', 36),
  ('American Council of Learned Socie

## Get subject area links for each main funder

In [158]:
df.columns

Index(['Unnamed: 0', 'Publication Type', 'Authors', 'Book Authors',
       'Book Editors', 'Book Group Authors', 'Author Full Names',
       'Book Author Full Names', 'Group Authors', 'Article Title',
       'Source Title', 'Book Series Title', 'Book Series Subtitle', 'Language',
       'Document Type', 'Conference Title', 'Conference Date',
       'Conference Location', 'Conference Sponsor', 'Conference Host',
       'Author Keywords', 'Keywords Plus', 'Abstract', 'Addresses',
       'Reprint Addresses', 'Email Addresses', 'Researcher Ids', 'ORCIDs',
       'Funding Orgs', 'Funding Text', 'Cited References',
       'Cited Reference Count', 'Times Cited, WoS Core',
       'Times Cited, All Databases', '180 Day Usage Count',
       'Since 2013 Usage Count', 'Publisher', 'Publisher City',
       'Publisher Address', 'ISSN', 'eISSN', 'ISBN', 'Journal Abbreviation',
       'Journal ISO Abbreviation', 'Publication Date', 'Publication Year',
       'Volume', 'Issue', 'Part Number', 'Suppleme

In [168]:
df['WoS Categories'][0], df['Research Areas'][0]
df['WoS Categories'][120], df['Research Areas'][120]


('International Relations; Law; Political Science',
 'International Relations; Government & Law')

In [175]:
## make a dict with nodes starting from main funders to the subject areas

def make_subject_dict(df,subjects_col):
    wosDict = dict()
    for f in funders:
        wosDict[f] = {}
        d = df[df.funder_dataset == f]
        for index,row in d.iterrows():
            if str(row[subjects_col]) != "nan":
                for subj in str(row[subjects_col]).split(";"):
                    s = subj.strip()
                    if s not in wosDict[f]:
                        wosDict[f][s] = 1
                    else:
                        wosDict[f][s] +=1
    return(wosDict)

In [176]:
wosDict = make_subject_dict(df,"WoS Categories")

In [177]:
areasDict = make_subject_dict(df,"Research Areas")

In [178]:
areasDict

{'mellon': {'Agriculture': 67,
  'Computer Science': 91,
  'Geography': 47,
  'Physical Geography': 56,
  'Information Science & Library Science': 63,
  'Chemistry': 133,
  'Electrochemistry': 1,
  'Instruments & Instrumentation': 8,
  'Education & Educational Research': 54,
  'Philosophy': 9,
  'Plant Sciences': 264,
  'Cultural Studies': 18,
  'Architecture': 9,
  'Communication': 16,
  'Environmental Sciences & Ecology': 759,
  'History': 79,
  'Literature': 21,
  'Health Care Sciences & Services': 27,
  'History & Philosophy of Science': 36,
  'Anthropology': 97,
  'Archaeology': 70,
  'Psychology': 52,
  'Physics': 39,
  'Spectroscopy': 73,
  'Science & Technology - Other Topics': 214,
  'Neurosciences & Neurology': 52,
  'Evolutionary Biology': 91,
  'Medical Informatics': 5,
  'Religion': 20,
  'Government & Law': 52,
  'Linguistics': 31,
  'Area Studies': 42,
  'Marine & Freshwater Biology': 142,
  'Toxicology': 6,
  'Business & Economics': 38,
  'Asian Studies': 17,
  'Arts & 

### association mining for subject areas and main funders

In [218]:
numPubs = {}
for f in funders:
    d = df[df.funder_dataset == f]
    numPubs[f] = len(d.index)

In [220]:

def get_subj_funder_associations(areasDict):
    subjectsDict = {}
    for k,v in areasDict.items():
        for sub,val in v.items():
            if sub not in subjectsDict:
                subjectsDict[sub] = val
            else:
                subjectsDict[sub] += val


    subjectsDf = pd.DataFrame()
    subjectsDf["subject"] = list(subjectsDict.keys())
    subjectsDf["total_freq"] = list(subjectsDict.values())
    for f in funders:
        subjectsDf[f+"_total_occ"] = 0
    for index,row in subjectsDf.iterrows():
        for f in funders:
            if row["subject"] in areasDict[f]: 
                ## supply the number of occurences
                subjectsDf.at[index,f+"_total_occ"] = areasDict[f][row["subject"]] 

    ## divide occurences by teh number of pubs in the dataset for the funder            
    for f in funders:
        subjectsDf[f+"_pct_of_pubs"] = subjectsDf[f+"_total_occ"] / numPubs[f]
        
    return(subjectsDf)
        
    



In [223]:
researchAreasAssociations = get_subj_funder_associations(areasDict)
researchAreasAssociations

,subject,total_freq,mellon_total_occ,packard_total_occ,gates_total_occ,osf_total_occ,rwj_total_occ,hewlett_total_occ,ford_total_occ,mellon_pct_of_pubs,packard_pct_of_pubs,gates_pct_of_pubs,osf_pct_of_pubs,rwj_pct_of_pubs,hewlett_pct_of_pubs,ford_pct_of_pubs
0,Agriculture,1232,67,20,1077,7,14,10,37,0.023484,0.002489,0.042069,0.006667,0.001530,0.009643,0.015969
1,Computer Science,744,91,227,218,21,92,70,25,0.031896,0.028251,0.008515,0.020000,0.010055,0.067502,0.010790
2,Geography,148,47,9,27,17,11,5,32,0.016474,0.001120,0.001055,0.016190,0.001202,0.004822,0.013811
3,Physical Geography,112,56,25,14,1,5,2,9,0.019628,0.003111,0.000547,0.000952,0.000546,0.001929,0.003884
4,Information Science & Library Science,148,63,3,28,7,35,6,6,0.022082,0.000373,0.001094,0.006667,0.003825,0.005786,0.002590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Integrative & Complementary Medicine,14,0,1,5,0,4,1,3,0.000000,0.000124,0.000195,0.000000,0.000437,0.000964,0.001295
148,Thermodynamics,26,0,2,17,2,0,0,5,0.000000,0.000249,0.000664,0.001905,0.000000,0.000000,0.002158
149,Emergency Medicine,109,0,0,5,3,100,0,1,0.000000,0.000000,0.000195,0.002857,0.010929,0.000000,0.000432
150,"Dentistry, Oral Surgery & Medicine",58,0,0,11,1,45,0,1,0.000000,0.000000,0.000430,0.000952,0.004918,0.000000,0.000432


In [224]:
wosCategoryAssociations = get_subj_funder_associations(wosDict)

In [225]:
wosCategoryAssociations    


,subject,total_freq,mellon_total_occ,packard_total_occ,gates_total_occ,osf_total_occ,rwj_total_occ,hewlett_total_occ,ford_total_occ,mellon_pct_of_pubs,packard_pct_of_pubs,gates_pct_of_pubs,osf_pct_of_pubs,rwj_pct_of_pubs,hewlett_pct_of_pubs,ford_pct_of_pubs
0,Soil Science,144,58,7,73,0,0,0,6,0.020329,0.000871,0.002851,0.000000,0.000000,0.000000,0.002590
1,"Computer Science, Information Systems",253,42,59,55,5,53,28,11,0.014721,0.007343,0.002148,0.004762,0.005792,0.027001,0.004748
2,Geography,148,47,9,27,17,11,5,32,0.016474,0.001120,0.001055,0.016190,0.001202,0.004822,0.013811
3,"Geography, Physical",112,56,25,14,1,5,2,9,0.019628,0.003111,0.000547,0.000952,0.000546,0.001929,0.003884
4,Information Science & Library Science,148,63,3,28,7,35,6,6,0.022082,0.000373,0.001094,0.006667,0.003825,0.005786,0.002590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,Emergency Medicine,109,0,0,5,3,100,0,1,0.000000,0.000000,0.000195,0.002857,0.010929,0.000000,0.000432
243,"Psychology, Mathematical",6,0,0,4,0,0,0,2,0.000000,0.000000,0.000156,0.000000,0.000000,0.000000,0.000863
244,"Dentistry, Oral Surgery & Medicine",58,0,0,11,1,45,0,1,0.000000,0.000000,0.000430,0.000952,0.004918,0.000000,0.000432
245,"Materials Science, Composites",2,0,0,1,0,1,0,0,0.000000,0.000000,0.000039,0.000000,0.000109,0.000000,0.000000


In [226]:
wosCategoryAssociations.to_csv("wosCategory_funder_associations.csv")

In [227]:
researchAreasAssociations.to_csv("researchAreas_funder_associations.csv")

## Make Cofunder <> Subject area links

In [184]:
## make cofunder subjects dict with nodes going from cofunders to subject areas

def make_cofunder_subject_links_dict(df,validated_cof_col,subjects_col):
    cofSubDict = dict()
    for f in funders:
        cofSubDict[f] = {}
        d = df[df.funder_dataset == f]
        for index,row in d.iterrows():
            ## if we have a list of cofunders
            if row[validated_cof_col] != "":
                ## for each cof
                for cof in row[validated_cof_col]:
                    ## make an entry
                    if cof not in cofSubDict[f]:
                        cofSubDict[f][cof] = {}
                    # for every row with a list of subjects
                    if str(row[subjects_col]) != "nan":
                        ## for each subject, add it to the links dict for the funder and cofunder
                        for subj in str(row[subjects_col]).split(";"):
                            s = subj.strip()
                            if s not in cofSubDict[f][cof]:
                                cofSubDict[f][cof][s] = 1
                            else:
                                cofSubDict[f][cof][s] +=1
    return(cofSubDict)

In [187]:
cofAreasDict = make_cofunder_subject_links_dict(df,'twice_validated_cof_list',"Research Areas")

In [188]:
cofWosSubDict = make_cofunder_subject_links_dict(df,'twice_validated_cof_list',"WoS Categories")

### write out data assets to json


In [190]:
with open('wosCategories.json', 'w') as outfile:
    json.dump(wosDict, outfile)
    
with open('researchAreas.json', 'w') as outfile:
    json.dump(areasDict, outfile)
    
with open('topCofunders.json', 'w') as outfile:
    json.dump(topCof_twiceValidated, outfile) 

with open('cofunder_researchArea_links.json', 'w') as outfile:
    json.dump(cofAreasDict, outfile)
    
with open('cofunder_wosCategories_links.json', 'w') as outfile:
    json.dump(cofWosSubDict, outfile) 

# Analyses

## to do:
### - Coverage of the top 10 areas for each funder; disjunctive probabilities
### - Split bar chart in d3 (See datawrapper for example; add sort feature; add Cofunder Stats on hoverover, add pub listings)
### - Sankey diagram in d3 
